In [11]:
import pandas as pd
import re

texto = pd.read_csv("C://Universidad//Cuarto//DCDC//EURES_TRADUCIDO.csv", sep=";")
patrón = re.compile(r"(\/[ao][s]?)([^A-Za-zÑñ])")
nuevo = lambda x: patrón.sub(r"\2", x)


In [12]:
texto["descripcion"] = texto["descripcion"].apply(nuevo)

In [13]:
texto.to_csv("C://Universidad//Cuarto//DCDC//EURES_TRADUCIDO.csv", sep=";", index=False)

In [14]:
texto["descripcion"] = texto["descripcion"].str.lower()

In [15]:
texto.to_csv("C://Universidad//Cuarto//DCDC//EURES_TRADUCIDO.csv", sep=";", index=False)

In [19]:
import re
import pandas as pd
import spacy

# cargar spaCy en español
nlp = spacy.load("es_core_news_sm")

# Definimos una clase de caracteres de puntuación explícita
PUNCTUACION = r"[.,;:()\[\]{}¿?!¡/\"'_\-«»…%]"

def limpiar_texto(texto):
    if pd.isna(texto):
        return ""
    # 1. Pasar a minúsculas
    t = texto.lower()
    # 2. Eliminar números
    t = re.sub(r"\d+", "", t)
    # 3. Eliminar signos de puntuación definidos
    t = re.sub(PUNCTUACION, " ", t)
    # 4. Eliminar espacios múltiples
    t = re.sub(r"\s+", " ", t).strip()
    return t

def procesar(texto):
    texto = limpiar_texto(texto)
    # tokenización + lematización con spaCy
    doc = nlp(texto)
    tokens = [token.lemma_ for token in doc if not token.is_stop and token.lemma_ != ""]
    return " ".join(tokens)

OSError: [E050] Can't find model 'es_core_news_sm'. It doesn't seem to be a Python package or a valid path to a data directory.

In [21]:
import re
import pandas as pd
import spacy

# Carga modelo de español de spaCy
# Si quieres más precisión: usa "es_core_news_md"
nlp = spacy.load("es_core_news_sm", disable=["ner", "parser"])

# Ruta del archivo
ruta = "C://Universidad//Cuarto//DCDC//EURES_TRADUCIDO.csv"

# Leer CSV (ajusta encoding si fuera necesario, por ejemplo encoding="utf-8")
df = pd.read_csv(ruta, sep=";")

# -------------------------------------------------------------------------
# REGEX para eliminar solo marcas /a, /as, /o, /os sin tocar /año o /agosto
# -------------------------------------------------------------------------
RE_GENERO = re.compile(r'(\b\w+)\s*/[ao]s?(?!\w)', flags=re.UNICODE)

# Clase de puntuación SIN la barra (la borramos más adelante)
PUNCT_SIN_SLASH = r"[.,;:()\[\]{}¿?!¡\"'_\-«»…%]"

def quitar_marcas_genero(texto: str) -> str:
    """Elimina /a, /as, /o, /os solo si son marcas de género."""
    return RE_GENERO.sub(r'\1', texto)

def limpiar_texto(texto):
    """Limpieza completa previa a la lematización."""
    if pd.isna(texto):
        return ""
    t = str(texto).lower()  # a minúsculas
    t = quitar_marcas_genero(t)  # quitar /a, /as, /o, /os
    t = re.sub(r"\d+", " ", t)  # quitar números
    t = re.sub(PUNCT_SIN_SLASH, " ", t)  # quitar puntuación (sin la barra)
    t = re.sub(r"/", " ", t)  # quitar barras restantes
    t = re.sub(r"\s+", " ", t).strip()  # quitar espacios múltiples
    return t

def procesar(texto):
    """Lematiza y elimina stopwords con spaCy."""
    t = limpiar_texto(texto)
    doc = nlp(t)
    tokens = [tok.lemma_ for tok in doc if not tok.is_stop and tok.lemma_]
    return " ".join(tokens)

# -------------------------------------------------------------------------
# Aplicar al CSV
# -------------------------------------------------------------------------

# Si tu CSV tiene una columna llamada "descripcion"
df["descripcion_proc"] = df["descripcion"].apply(procesar)

# -------------------------------------------------------------------------
# Guardar resultado en un nuevo CSV
# -------------------------------------------------------------------------
df.to_csv("C://Universidad//Cuarto//DCDC//EURES_TRADUCIDO_PROC.csv", sep=";", index=False, encoding="utf-8-sig")

print("✅ Archivo procesado guardado como EURES_TRADUCIDO_PROC.csv")


✅ Archivo procesado guardado como EURES_TRADUCIDO_PROC.csv
